# Python Refresher Lab — FAANG-Level Hands-On

**Goal:** Refresh Python fundamentals + interview patterns the way FAANG expects ML engineers to code.

**Outcome:** You can write clean, efficient Python, explain complexity, and avoid classic interview gotchas.

---

## How to Start
1. Open this notebook in Colab
2. Complete all **TODO** sections
3. Restart runtime → Run all

---

## Lab Rules (FAANG Style)
- ✅ Complexity for each core function
- ✅ Handle edge cases
- ✅ Prefer O(n) over O(n²) when feasible
- ✅ Add tests
- ❌ No hardcoding outputs


In [1]:
from __future__ import annotations
from collections import Counter, defaultdict, deque
import heapq

# NOTE: Use this helper for quick sanity checks
def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

## Section 1 — Python Idioms & Core Data Structures

### Task 1.1: Comprehensions (no unnecessary loops)

Create (without loops):
- `even_squares`: squares of even numbers from 1..20
- `pairs`: list of (x, x*x) for x=1..10
- `flattened`: flatten `nested`

**Explain:** Why can comprehensions be faster than manual loops?

In [2]:
# TODO:
# TC: O(n) where n is length of numbers
# Create even_squares
# HINT: range(1, 21), filter even with x % 2 == 0
even_squares = [x*x for x in range(1,21) if x%2 == 0]

# TODO:
# TC: O(n) where n is length of numbers
# Create pairs
pairs = [(x, x*x) for x in range(1,11)]

# TODO:
# TC: O(n) where n is length of numbers
# Flatten nested
nested = [[1, 2], [3, 4], [5, 6]]
# HINT: nested loops inside a single comprehension
flattened = [element for row in nested for element in row]

check('even_squares', even_squares == [4, 16, 36, 64, 100, 144, 196, 256, 324, 400])
check('pairs', pairs[:3] == [(1, 1), (2, 4), (3, 9)] and pairs[-1] == (10, 100))
check('flattened', flattened == [1, 2, 3, 4, 5, 6])

OK: even_squares
OK: pairs
OK: flattened


### Task 1.2: Counter (frequency counting)

Given `s`, compute:
- character frequency using `Counter`
- top 2 most common characters

**Interview angle:** When is Counter preferable to sorting?

In [3]:
s = 'mississippi'

# TODO:
# TC: O(n) where n is length of s
freq = Counter(s)
top2 = freq.most_common(2)

print(freq)
print(top2)
check('top2', top2[0][0] == 'i' and top2[0][1] == 4)

Counter({'i': 4, 's': 4, 'p': 2, 'm': 1})
[('i', 4), ('s', 4)]
OK: top2


### Task 1.3: defaultdict (grouping)

Group words by first letter.

**HINT:** `defaultdict(list)`

**Gotcha:** Don’t use a normal dict unless you handle missing keys.

In [4]:
words = ['apple', 'apricot', 'banana', 'blueberry', 'cherry', 'avocado']

# TODO:
# TC: O(n) where n is length of words
groups = defaultdict(list)
for word in words:
  letter = word[0]
  groups[letter].append(word)
print(dict(groups))
check('groups[a]', dict(groups)['a'] == ['apple', 'apricot', 'avocado'])

{'a': ['apple', 'apricot', 'avocado'], 'b': ['banana', 'blueberry'], 'c': ['cherry']}
OK: groups[a]


## Section 2 — FAANG Core Patterns

### Task 2.1: Two Sum (Hash Map Pattern)

**FAANG Classic.** Return indices of two numbers that sum to target.

# HINT:
- Use a dict: value -> index
- For each `x`, check if `target-x` has been seen

**Explain:** Why is this O(n) time?

In [5]:
def two_sum(nums, target):
    # TimeComplexity: O(n) where n is the length of nums
    # SpaceComplexity: O(n) where n is the length of nums
    # Reason: Because we are going through all the elements in nums it would be O(n)
    hashMap = {}
    for index, value in enumerate(nums):
      difference = target - value
      if difference in hashMap:
        return [hashMap[difference], index]
      hashMap[value] = index
    return []

check('two_sum_1', two_sum([2, 7, 11, 15], 9) == [0, 1])
check('two_sum_2', two_sum([3, 2, 4], 6) == [1, 2])
check('two_sum_dup', two_sum([3, 3], 6) == [0, 1])

OK: two_sum_1
OK: two_sum_2
OK: two_sum_dup


### Task 2.2: Longest Substring Without Repeating Characters (Sliding Window)

**FAANG Classic/Gotcha:** naive O(n²) solutions time out.

# HINT:
- Keep a map char -> most recent index
- Maintain a window `[l..r]` with unique chars

**Explain:** What invariant does the window maintain?

In [6]:
def length_of_longest_substring(s: str) -> int:
    # TimeComplexity: O(n) where n is length of the given string.
    # SpaceComplexity: O(n) where n is length of the given string.
    # Answer: The window helps us maintain the unique characters in the substring
    start = 0
    maxLength = 0
    hashMap = {}
    for index, char in enumerate(s):
      if char in hashMap:
        start = max(start, hashMap[char])
      maxLength = max(maxLength, index - start + 1)
      hashMap[char] = index + 1
    return maxLength

check('ls_abcabcbb', length_of_longest_substring('abcabcbb') == 3)
check('ls_bbbbb', length_of_longest_substring('bbbbb') == 1)
check('ls_pwwkew', length_of_longest_substring('pwwkew') == 3)
check('ls_empty', length_of_longest_substring('') == 0)

OK: ls_abcabcbb
OK: ls_bbbbb
OK: ls_pwwkew
OK: ls_empty


### Task 2.3: Product of Array Except Self (Prefix/Suffix)

**FAANG Classic/Gotcha:**
- No division
- O(n) time

# HINT:
- Build prefix products in one pass
- Multiply by suffix products in reverse pass

**Explain:** Why does this work with zeros?

In [7]:
def product_except_self(nums):
    # TimeComplexity: O(n) where n is the length of the nums array. We have two passes so it would be 2O(n) but we ignore constants so O(n)
    # Space Complexity: O(n) where n is the length of the result array.
    # Reason: This works with zeros because we do the product of everything except self and we do not use the division method.
    n = len(nums)
    result = [1] * n
    runningProduct = 1
    for i in range(1, n):
      runningProduct *= nums[i - 1]
      result[i] = runningProduct
    runningProduct = 1
    for i in range(n - 2, -1, -1):
      runningProduct *= nums[i + 1]
      result[i] = result[i] * runningProduct
    return result

check('pes_1234', product_except_self([1, 2, 3, 4]) == [24, 12, 8, 6])
check('pes_zero', product_except_self([-1, 1, 0, -3, 3]) == [0, 0, 9, 0, 0])

OK: pes_1234
OK: pes_zero


## Section 3 — FAANG Hard (Optional / Bonus)

### Task 3.1: Sliding Window Maximum (Deque)

**FAANG Classic:** maintain a decreasing deque of indices.

# HINT:
- Pop from left if out of window
- Pop from right while current value is larger

**Explain:** Why is this amortized O(n)?

In [8]:
def max_sliding_window(nums, k):
    # So we move in window of the given length and maintain a deque of all the elements greater than the current element. We pop the elements
    # on the right till the current value is larger. To keep the window size same we pop the elements from the left of the deque since we know
    # the left most element would be the largest in that window
    # TC: O(n) + O(k) but we can consider amortized O(n) since we know k is 3 and since it is a constant we can ignore it.
    # SC: O(n) where n is the length of the result array
    dq = deque()
    result = []
    for i in range(k):
      while dq and nums[i] >= nums[dq[-1]]:
        dq.pop()
      dq.append(i)
    result.append(nums[dq[0]])
    for i in range(k, len(nums)):
      while dq and nums[i] >= nums[dq[-1]]:
        dq.pop()
      if dq and dq[0] <= i - k:
        dq.popleft()
      dq.append(i)
      result.append(nums[dq[0]])
    return result

check('msw', max_sliding_window([1,3,-1,-3,5,3,6,7], 3) == [3,3,5,5,6,7])

OK: msw


### Task 3.2: LRU Cache (Hash Map + Doubly Linked List)

**FAANG Hard/Gotcha:** You must get O(1) `get` and `put`.

# HINT:
- Dict: key -> node
- Doubly linked list: move-to-front on access

You can implement this later if you run out of time today.

In [9]:
class Node:
    def __init__(self):
        self.key = 0
        self.value = 0
        self.prev = None
        self.next = None
class LRUCache:
    def __init__(self, capacity: int):
        self.head = Node()
        self.tail = Node()
        self.capacity = capacity
        self.map = {}
        self.head.next = self.tail
        self.tail.prev = self.head
    def addToHead(self, node):
        node.next = self.head.next
        node.prev = self.head
        node.next.prev = node
        self.head.next = node
    def removeNode(self, node):
        node.next.prev = node.prev
        node.prev.next = node.next
    def get(self, key: int) -> int:
      # TC: O(1) since we are not going over all the elements and we are using hashmap for faster retrieval
      # SC: O(n) where n is equal to capacity
        if key in self.map:
            node = self.map[key]
            self.removeNode(node)
            self.addToHead(node)
            return node.value
        return -1
    def put(self, key: int, value: int) -> None:
      # TC: O(1) since we are not going over all the elements and we are using hashmap for faster retrieval
      # SC: O(n) where n is equal to capacity
        if key in self.map:
            node = self.map[key]
            node.value = value
            self.removeNode(node)
            self.addToHead(node)
        else:
            if self.capacity == len(self.map):
                tailPrev = self.tail.prev
                self.removeNode(tailPrev)
                del self.map[tailPrev.key]
            newNode = Node()
            newNode.key = key
            newNode.value = value
            self.map[key] = newNode
            self.addToHead(newNode)

cache = LRUCache(2)
cache.put(1, 1)
cache.put(2, 2)
check('lru_1', cache.get(1) == 1)
cache.put(3, 6)
check('lru_2', cache.get(2) == -1)
cache.put(4, 7)
check('lru_3', cache.get(3) == 6)

OK: lru_1
OK: lru_2
OK: lru_3


---

## Submission Checklist
- All TODOs completed
- All tests passing
- Explain prompts answered
- Complexity noted
